In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-3.0.1-py3-none-any.whl.metadata (10.0 kB)
Using cached pybind11-3.0.1-py3-none-any.whl (293 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp312-cp312-linux_x86_64.whl size=4498211 sha256=773e467e67d7d51a65c5b8b816debcbe98038f045e2c7cdcdaef1ae08f4a4e48
  Stored in directory: /root/.cache/pip/wheels/20/27/95/a7baf1b435f1cbde017cabdf1e9688526d2b0e929255a359c6
Successfully built fasttext


In [ ]:
!pip uninstall numpy -y
!pip install numpy==1.26.4

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 78.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import fasttext
import numpy as np

In [ ]:
train = "/content/train_en_yo_pcm.txt"

In [ ]:
!ls /content/train_en_yo_pcm.txt

/content/train_en_yo_pcm.txt


In [ ]:
model = fasttext.train_supervised(
    input=train,
    lr=0.5,
    epoch=25,
    wordNgrams=2,
    verbose=2
)

In [ ]:
# Quick accuracy check
def predict_language_safe(model, text):
    # Handle empty or invalid input
    if not isinstance(text, str) or not text.strip():
        return "unknown", 0.0

    labels, probs = model.predict(text)

    # Handle case when FastText returns nothing
    if not labels or not probs:
        return "unknown", 0.0

    label = labels[0].replace("__label__", "")
    prob = float(probs[0])
    return label, prob


# Try predictions
print("\n--- Testing Prediction model v1 ---")

samples = ["I wan go buy food", "Bawo ni", "Please send airtime"]

for s in samples:
    print(s, "→", model.predict(s))

print("\n--- Testing Prediction model v2 ---")
for s in samples:
    label, prob = predict_language_safe(model, s)
    print(f"'{s}' → Label: {label} (Prob: {prob:.4f})")


--- Testing Prediction model v1 ---
I wan go buy food → (('__label__pcm',), array([1.00000989]))
Bawo ni → (('__label__yo',), array([0.86500198]))
Please send airtime → (('__label__pcm',), array([0.84117138]))

--- Testing Prediction model v2 ---
'I wan go buy food' → Label: pcm (Prob: 1.0000)
'Bawo ni' → Label: yo (Prob: 0.8650)
'Please send airtime' → Label: pcm (Prob: 0.8412)


In [ ]:
model.save_model("language_detect_uncompressed.ftz")
model.quantize(input=train, retrain=True, cutoff=100000)
model.save_model("language_detect.ftz")

In [63]:
from google.colab import files
#files.download("language_detect.ftz")
files.download("language_detect_uncompressed.ftz")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>